In [1]:
from abc import ABCMeta, abstractmethod,abstractproperty
import codecs
import json
import sys
import pymongo
from pymongo import MongoClient

In [2]:
def progress(ind,size):
    percent = int(100*float(ind)/size)
    sys.stdout.write('\r[{0}{1}] {2}% {3}'.format('#'*(percent/10),' '*(10-percent/10), percent, ind))
    sys.stdout.flush()

In [9]:
class DocumentIter(object):
    __metaclass__ = ABCMeta
    
    @abstractproperty
    def source():
        return None
    
    @abstractproperty
    def size():
        return None
    
    @abstractmethod
    def __iter__():
        yield None
        
    @abstractproperty    
    def sanitise():
        return None
        
class OneLineDiskDocIter(DocumentIter):
    size=0
    source=''
    sanitiser=None

    def __init__(self,txf,sanit):
        self.source=txf
        self.sanitiser=sanit
        ind=0
        for line in codecs.open(txf,'r',encoding='utf8'):
            ind+=1
        self.size=ind

    def __iter__(self):
        for line in codecs.open(self.source,'r',encoding='utf8'):
            if ind%1000==0:
                progress(ind,self.size)
            doc = self.sanitiser.sanitise(line).split()
            yield doc
            
class MultiLineDiskDocIter(DocumentIter):
    size=0
    source=''
    sanitiser=None
    
    def __init__(self,txf,sanit):
        self.text_file=txf
        self.sanitiser=sanit
        ind=0
        for line in codecs.open(txf,'r',encoding='utf8'):
            ind+=1
        self.size=ind
        
    def __iter__(self):
        ind=0
        for line in codecs.open(self.source,'r',encoding='utf8'):
            record = json.loads(line)
            title = record['title']
            abstract_sents = record['abstract']
            ind+=1
            doc=[]
            if ind%1000==0:
                progress(ind,self.size)
            doc+=self.sanitiser.sanitise(title).split()
            for sent in abstract_sents:
                doc+=self.sanitiser.sanitise(sent).split()
            yield doc
        print('\n')
        
class MultiLineDbDocIter(DocumentIter):
    size=0
    source=''
    sanitiser=None
    
    def __init__(self,db_conn,query,sanit):
        self.sanitiser=sanit
        self.source=MongoClient(db_conn)
        self.query=query
        self.size=self.source.find(query).count()
    
    def __iter__(self):
        ind=0
        for record in self.source.find(self.query):
            title = record['title']
            abstract_sents = record['abstract']
            ind+=1
            doc=[]
            if ind%1000==0:
                progress(ind,self.size)
            doc+=self.sanitiser.sanitise(title).split()
            for sent in abstract_sents:
                doc+=self.sanitise.sanitise(sent).split()
            yield doc
        print('\n')
        
class MultiLineDbSentIter(DocumentIter):
    size=0
    source=''
    sanitiser=None
    
    def __init__(self,db_conn,query,sanit):
        self.sanitiser = sanit
        self.source=MongoClient(db_conn)
        self.query=query
        self.size=self.source.find(query).count()
    
    def __iter__(self):
        ind=0
        for record in self.source.find(self.query):
            title = record['title']
            abstract_sents = record['abstract']
            ind+=1
            if ind%1000==0:
                progress(ind,self.size)
            yield self.sanitiser.sanitise(title).split()
            for sent in abstract_sents:
                yield self.sanitiser.sanitise(sent).split()
        print('\n')
            
class OneLineDbSentIter(DocumentIter):
    size=0
    source=''
    sanitiser=None
    
    def __init__(self,db_conn,query,sanit):
        self.source=MongoClient(db_conn)
        self.query=query
        self.size=self.source.find(query).count()
        self.sanitiser=sanit
    
    def __iter__(self):
        ind=0
        for record in self.source.find(self.query):
            title = record['title']
            ind+=1
            if ind%1000==0:
                progress(ind,self.size)
            yield self.sanitiser.sanitise(title).split()
        print('\n')
        
class OneLineMemorySentIter(DocumentIter):
    size =0
    source=[]
    sanitiser=None
    
    def __init__(self,source,sanit):
        self.size = len(source)
        self.source=source
        self.sanitser=sanit
        
    def __iter(self):
        ind=0
        for record in self.source:
            ind+=1
            if ind%1000==0:
                progress(ind,self.size)
            yield self.sanitiser(record).split()
        print('\n')


            
    

In [7]:
a = OneLine_disk_document_iter('bible.txt',lambda x:x)
#b = OneLine_db_document_iter()
#c = MultiLine_db_document_iter()
#d = MultiLine_db_sentence_iter()
e = MultiLine_disk_document_iter('exp2.json',lambda x:x)

In [ ]:
for i in iterator
    funct(i)